#GANSPACE - Discovering Interpretable GAN Controls 

Using https://github.com/harskish/ganspace to find latent directions in a stylegan2 model. (This could easily be ported to other models, if anyone implements it please get in touch, and i'll add it to the notebook!)

Notebook put together by [@realmeatyhuman](https://twitter.com/realmeatyhuman)



In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/



## Setup
Hit play on all the cells below, and everything should run smoothly. The install takes around a minute and a half - go grab yourself a cup of tea.



In [ ]:
%cd /content/
!git clone https://github.com/woctezuma/ganspace

%cd /content/ganspace/
!git checkout steam

/content
Cloning into 'ganspace'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 309 (delta 12), reused 75 (delta 10), pack-reused 223
Receiving objects: 100% (309/309), 46.38 MiB | 49.02 MiB/s, done.
Resolving deltas: 100% (85/85), done.
/content/ganspace
Branch 'steam' set up to track remote branch 'steam' from 'origin'.
Switched to a new branch 'steam'


In [ ]:
#@title Install remaining packages
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 200})'''))

!pip install fbpca pyopengltk Ninja
!git submodule update --init --recursive
!python -c "import nltk; nltk.download('wordnet')"
%cd models/stylegan2/stylegan2-pytorch/op
!python setup.py install
!python -c "import torch; import upfirdn2d_op; import fused; print('OK')"
%cd "/content/ganspace"

<IPython.core.display.Javascript object>

Submodule 'stylegan/stylegan_tf' (https://github.com/NVlabs/stylegan.git) registered for path 'models/stylegan/stylegan_tf'
Submodule 'stylegan2/stylegan2-pytorch' (https://github.com/harskish/stylegan2-pytorch.git) registered for path 'models/stylegan2/stylegan2-pytorch'
Cloning into '/content/ganspace/models/stylegan/stylegan_tf'...
Cloning into '/content/ganspace/models/stylegan2/stylegan2-pytorch'...
Submodule path 'models/stylegan/stylegan_tf': checked out '66813a32aac5045fcde72751522a0c0ba963f6f2'
Submodule path 'models/stylegan2/stylegan2-pytorch': checked out '76ca1058fc587b1846523bc382f0ece848bb0711'
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/content/ganspace/models/stylegan2/stylegan2-pytorch/op
running install
running bdist_egg
running egg_info
creating upfirdn2d.egg-info
writing upfirdn2d.egg-info/PKG-INFO
writing dependency_links to upfirdn2d.egg-info/dependency_links.txt
writing top-level names to up

# Convert model weights

If you have a tensorflow model you want to use Ganspace on - convert it to a pytorch model below.

(skip this step if you already have a pytorch model)

In [ ]:
# Reference: https://drive.google.com/file/d/1HXczmPE4PMBbhrPOshshnS0KFI4J6jbC/view?usp=sharing
!gdown --id 1HXczmPE4PMBbhrPOshshnS0KFI4J6jbC -O /content/steam.pkl

Downloading...
From: https://drive.google.com/uc?id=1HXczmPE4PMBbhrPOshshnS0KFI4J6jbC
To: /content/steam.pkl
296MB [00:02, 107MB/s]


In [ ]:
%cd '/content/'
!git clone https://github.com/skyflynil/stylegan2

/content
Cloning into 'stylegan2'...
remote: Enumerating objects: 219, done.
remote: Total 219 (delta 0), reused 0 (delta 0), pack-reused 219
Receiving objects: 100% (219/219), 2.29 MiB | 30.09 MiB/s, done.
Resolving deltas: 100% (120/120), done.


The convert weight script takes two arguments: 

```
--repo - Path to tensorflow stylegan2 repo
       - Path to your model
```



In [ ]:
%cd /content/ganspace/
!python models/stylegan2/stylegan2-pytorch/convert_weight.py --repo="/content/stylegan2/" --channel_multiplier=1 "/content/steam.pkl" #convert weights

/content/ganspace
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.


# Run PCA Analysis

In [ ]:
%cd /content/ganspace/

/content/ganspace


In [ ]:
model_name = 'StyleGAN2' 
model_class = 'steam' #this is the name of your model in the configs
num_components = 80

In [ ]:
%cp /content/ganspace/steam.pt /content/ganspace/models/checkpoints/stylegan2/stylegan2_steam_256.pt

Reference: https://github.com/harskish/ganspace

##Options


```
Command line paramaters:
  --model      one of [ProGAN, BigGAN-512, BigGAN-256, BigGAN-128, StyleGAN, StyleGAN2]
  --class      class name; leave empty to list options
  --layer      layer at which to perform PCA; leave empty to list options
  --use_w      treat W as the main latent space (StyleGAN / StyleGAN2)
  --inputs     load previously exported edits from directory
  --sigma      number of stdevs to use in visualize.py
  -n           number of PCA samples
  -b           override automatic minibatch size detection
  -c           number of components to keep

```




In [ ]:
#Check layers available for analysis
!python visualize.py --model $model_name --class $model_class --use_w

[21.09 18:42] StyleGAN2, g_mapping, ipca
Layer 'g_mapping' not found in model!
Available layers: 
style
style.0
style.1
style.2
style.3
style.4
style.5
style.6
style.7
style.8
input
conv1
conv1.conv
conv1.conv.modulation
conv1.noise
conv1.activate
to_rgb1
to_rgb1.conv
to_rgb1.conv.modulation
convs
convs.0
convs.0.conv
convs.0.conv.blur
convs.0.conv.modulation
convs.0.noise
convs.0.activate
convs.1
convs.1.conv
convs.1.conv.modulation
convs.1.noise
convs.1.activate
convs.2
convs.2.conv
convs.2.conv.blur
convs.2.conv.modulation
convs.2.noise
convs.2.activate
convs.3
convs.3.conv
convs.3.conv.modulation
convs.3.noise
convs.3.activate
convs.4
convs.4.conv
convs.4.conv.blur
convs.4.conv.modulation
convs.4.noise
convs.4.activate
convs.5
convs.5.conv
convs.5.conv.modulation
convs.5.noise
convs.5.activate
convs.6
convs.6.conv
convs.6.conv.blur
convs.6.conv.modulation
convs.6.noise
convs.6.activate
convs.7
convs.7.conv
convs.7.conv.modulation
convs.7.noise
convs.7.activate
convs.8
convs.8.conv


Add chosen layer in as --layer argument:

In [ ]:
!python visualize.py --model $model_name --class $model_class --use_w --layer=style -c $num_components

[21.09 17:41] StyleGAN2, style, ipca
Feature shape: torch.Size([1, 512])
Not cached
[21.09 17:41] Computing stylegan2-steam_style_ipca_c80_n300000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:260: FutureWarning: torch.cuda.reset_max_memory_cached now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:234: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  FutureWarning)
B=20, N=300000, dims=512, N/dims=585.9
Sampling latents: 100% 15100/15100 [00:30<00:00, 499.27it/s]
Fitting batches (NB=2000): 100% 150/150 [00:46<00:00,  3.25it/s]
Total time: 0:01:16.986872
/usr/local/lib/python3.6/dist-packages/torch/cuda/memory.py:260: FutureWarning: torch.cuda.reset_max_memory_cached now calls tor

In [ ]:
!python visualize.py --model=$model_name --class=$model_class --use_w --layer="style" -b=500 --batch --video #add -video to generate videos

[21.09 17:43] StyleGAN2, style, ipca
Feature shape: torch.Size([1, 512])
Batch size: 500
[21.09 17:43] Creating visualizations
Sparsity: 0.00
Random images: 100% 10/10 [11:00<00:00, 66.05s/it]
Done in 0:14:06.259958


In [ ]:
!python visualize.py --model=StyleGAN2 --class=ffhq --use_w --layer=style -b=10000

[21.09 17:57] StyleGAN2, style, ipca
Feature shape: torch.Size([1, 512])
Not cached
[21.09 17:58] Computing stylegan2-ffhq_style_ipca_c80_n300000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
B=10000, N=300000, dims=512, N/dims=585.9
Sampling latents: 100% 31/31 [00:07<00:00,  4.02it/s]
Fitting batches (NB=10000): 100% 30/30 [00:29<00:00,  1.02it/s]
Total time: 0:00:37.560743
Batch size: 10000
[21.09 17:58] Creating visualizations
Sparsity: 0.00
<Figure size 1400x1200 with 14 Axes>
<Figure size 1400x1200 with 14 Axes>
Random images:   0% 0/10 [00:00<?, ?it/s]<Figure size 1400x1200 with 14 Axes>
Random images:  10% 1/10 [00:18<02:45, 18.42s/it]<Figure size 1400x1200 with 14 Axes>
Random images:  20% 2/10 [00:37<02:28, 18.56s/it]<Figure size 1400x1200 with 14 Axes>
Random images:  30% 3/10 [00:55<02:09, 18.53s/it]<Figure size 1400x1200 with 14 Axes>
Random images:  40% 4/10 [01:13<01:50, 18.40s/it]<Figure size 1400x1200 with 14 Axes>
Ra

In [ ]:
!zip -r samples.zip "/content/ganspace/out/StyleGAN2-ffhq" #zip up samples for download

  adding: content/ganspace/out/StyleGAN2-ffhq/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/inst/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/comp/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/summ/ (stored 0%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/summ/samp1_real_W.jpg (deflated 3%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/summ/samp7_real_W.jpg (deflated 3%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/summ/samp2_real_W.jpg (deflated 4%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/summ/samp8_real_W.jpg (deflated 4%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/summ/samp3_real_W.jpg (deflated 4%)
  adding: content/ganspace/out/StyleGAN2-ffhq/style/ipca/summ/random_dirs_W.jpg (deflated 4%)
  adding: content/gans

In [ ]:
%cd /content/ganspace/
!tar -cf StyleGAN2-steam.tar.gz out/StyleGAN2-steam/

# Explore Directions!

In [ ]:
!python visualize.py \
 --model StyleGAN2 \
 --class steam \
 --layer style \
 --use_w \
 -n 1_000_000 \
 -b 10_000 \
 --batch \
 --video

[21.09 18:44] StyleGAN2, style, ipca
Feature shape: torch.Size([1, 512])
Not cached
[21.09 18:44] Computing stylegan2-steam_style_ipca_c80_n1000000_w.npz
Reusing InstrumentedModel instance
Using W latent space
Feature shape: torch.Size([1, 512])
B=10000, N=1000000, dims=512, N/dims=1953.1
tcmalloc: large alloc 2068480000 bytes == 0xbd358000 @  0x7fde4abcf001 0x7fddfe552765 0x7fddfe5b6bb0 0x7fddfe5b8a4f 0x7fddfe64f048 0x50a7f5 0x50cfd6 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x50b053 0x634dd2 0x634e87 0x63863f 0x6391e1 0x4b0dc0 0x7fde4a7cab97 0x5b26fa
Sampling latents: 100% 101/101 [00:27<00:00,  3.67it/s]
Fitting batches (NB=10000): 100% 100/100 [01:52<00:00,  1.13s/it]
Total time: 0:02:21.828287
Batch size: 10000
[21.09 18:47] Creating visualizations
Sparsity: 0.00
Random images: 100% 10/10 [11:52<00:00, 71.29s/it]
Done in 0:17:37.030068


In [ ]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 5.3MB/s 
     |████████████████████████████████| 71kB 1.8MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
  Created wheel for pycuda: filename=pycuda-2019.1.2-cp36-cp36m-linux_x86_64.whl size=4535399 sha256=f53968ba01e256c935e3a8007a58d7d4fb026381bba521352d1c34e9c04519ea
  Stored in directory: /root/.cache/pip/wheels/a6/60/f0/b1c430c73d281ac3e46070480db50f7907364eb6f6d3188396
  Created wheel for pytools: filename=pytools-2020.4-py2.py3-none-any.whl size=67175 sha256=71393f988b02bccc19465da25a10f0b809238e356ee22b0ddf4449d82efeddfb
  Stored in directory: /root/.cache/pip/wheels/5e/20/0b/fac51840734b2587ecc239a62522b164c374e929e2c9be66c5
Successfully built pycuda pytools


```
Command line paramaters:
  --model      one of [ProGAN, BigGAN-512, BigGAN-256, BigGAN-128, StyleGAN, StyleGAN2]
  --class      class name; leave empty to list options
  --layer      layer at which to perform PCA; leave empty to list options
  --use_w      treat W as the main latent space (StyleGAN / StyleGAN2)
  --inputs     load previously exported edits from directory
  --sigma      number of stdevs to use in visualize.py
  -n           number of PCA samples
  -b           override automatic minibatch size detection
  -c           number of components to keep

```

After running visualize.py, your components will be stored in an npz file in /content/ganspace/cache/components/ - below the npz file is unpacked, and a component/direction is chosen at random. 

Using the UI, you can explore the latent direction and give it a name, which will be appeneded to the named_directions dictionary and saved as 'direction_name.npy' for later use.


In [ ]:
# Load model
from IPython.utils import io
import torch
import PIL
import numpy as np
import ipywidgets as widgets
from PIL import Image
import imageio
from models import get_instrumented_model
from decomposition import get_or_compute
from config import Config
from skimage import img_as_ubyte

# Speed up computation
torch.autograd.set_grad_enabled(False)
torch.backends.cudnn.benchmark = True

# Specify model to use
config = Config(
  model='StyleGAN2',
  output_class='steam',
  layer='style',
  use_w=True,
  components=80,
  batch_size=5_000, # style layer quite small
)

inst = get_instrumented_model(config.model, config.output_class,
                              config.layer, torch.device('cuda'), use_w=config.use_w)

path_to_components = get_or_compute(config, inst)

model = inst.model

named_directions = {} #init named_directions dict to save directions

In [ ]:
# Select a component at random
component_num = np.random.randint(num_components)
print(component_num)

8


In [ ]:
# or select one manually
component_num = 2

In [ ]:
#@title Load component

comps = np.load(path_to_components)
lst = comps.files
latent_dirs = []
latent_stdevs = []

load_activations = False

for item in lst:
    if load_activations:
      if item == 'act_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'act_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
    else:
      if item == 'lat_comp':
        for i in range(comps[item].shape[0]):
          latent_dirs.append(comps[item][i])
      if item == 'lat_stdev':
        for i in range(comps[item].shape[0]):
          latent_stdevs.append(comps[item][i])
            
#load one at random 
if component_num in named_directions.values():
  print(f'Direction already named: {list(named_directions.keys())[list(named_directions.values()).index(num)]}')

random_dir = latent_dirs[component_num]
random_dir_stdev = latent_stdevs[component_num]

print(f'Loaded Component No. {component_num}')


Loaded Component No. 2


In [ ]:
#@title Run UI
from ipywidgets import fixed

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

def name_direction(sender):
  if component_num in named_directions.values():
    target_key = list(named_directions.keys())[list(named_directions.values()).index(component_num)]
    print(f'Direction already named: {target_key}')
    print(f'Overwriting... ')
    del(named_directions[target_key])
  named_directions[text.value] = [component_num, start_layer.value, end_layer.value]
  save_direction(random_dir, text.value)
  for item in named_directions:
    print(item, named_directions[item])

def save_direction(direction, filename):
  dn = filename
  filename += f'_layers_{start_layer.value}_{end_layer.value}' + ".npy"
  filename = "out/directions/" + filename

  np.save(filename, direction, allow_pickle=True, fix_imports=True)
  print(f'Latent direction saved as {filename}')

def display_sample_pytorch(seed, truncation, direction, distance, scale, start, end, disp=True, save=None, noise_spec=None):
    # blockPrint()
    with io.capture_output() as captured:
      w = model.sample_latent(1, seed=seed).cpu().numpy()

      model.truncation = truncation
      w = [w]*model.get_max_latents() # one per layer
      for l in range(start, end):
          w[l] = w[l] + direction * distance * scale

      #save image and display
      out = model.sample_np(w)
      final_im = Image.fromarray((out * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS)

    if disp:
      display(final_im)
    if save is not None:
      if disp == False:
        print(save)
      final_im.save(f'out/{seed}_{save:05}.png')

def generate_mov(seed, truncation, direction_vec, scale, layers, n_frames, out_name = 'out', noise_spec = None, loop=True):
  """Generates a mov moving back and forth along the chosen direction vector"""
  # Example of reading a generated set of images, and storing as MP4.
  %mkdir out
  movieName = f'out/{out_name}.mp4'
  offset = -10
  step = 20 / n_frames
  imgs = []
  for i in log_progress(range(n_frames), name = "Generating frames"):
    print(f'\r{i} / {n_frames}', end='')
    w = model.sample_latent(1, seed=seed).cpu().numpy()

    model.truncation = truncation
    w = [w]*model.get_max_latents() # one per layer
    for l in layers:
      if l <= model.get_max_latents():
          w[l] = w[l] + direction_vec * offset * scale

    #save image and display
    out = model.sample_np(w)
    final_im = Image.fromarray((out * 255).astype(np.uint8))
    imgs.append(out)
    #increase offset
    offset += step
  if loop:
    imgs += imgs[::-1]
  with imageio.get_writer(movieName, mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(img_as_ubyte(image))


seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)
start_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=0, description='start layer: ', continuous_update=False)
end_layer = widgets.IntSlider(min=0, max=model.get_max_latents(), step=1, value=18, description='end layer: ', continuous_update=False)

# Make sure layer range is valid
def update_range_start(*args):
  end_layer.min = start_layer.value
def update_range_end(*args):
  start_layer.max = end_layer.value
start_layer.observe(update_range_start, 'value')
end_layer.observe(update_range_end, 'value')

text = widgets.Text(description="Name component here", style=style, width=200)

bot_box = widgets.HBox([seed, truncation, distance, scale, start_layer, end_layer, text])
ui = widgets.VBox([bot_box])

out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': start_layer, 'end': end_layer})

display(ui, out)
text.on_submit(name_direction)

Output()

In [ ]:
print(named_directions)

{}


In [ ]:
#script to generate a movie moving back and forth along the direction
direction_name = 'squint'
num_samples = 10
scale = 1 # scale the distance along the direction that the video shows
num_frames = 25 

loc = named_directions[direction_name][0]
for i in range(num_samples):
  s = np.random.randint(0, 10000)
  generate_mov(seed = s, truncation = 0.8, direction_vec = latent_dirs[loc], scale = scale, layers=range(named_directions[direction_name][1], named_directions[direction_name][2]), n_frames = num_frames, out_name = f'{model_class}_{direction_name}_{i}', loop=True)

KeyError: ignored

In [ ]:
#@title Select from named directions

from IPython.display import display, clear_output

vardict = list(named_directions.keys())
select_variable = widgets.Dropdown(
    options=vardict,
    value=vardict[0],
    description='Select variable:',
    disabled=False,
    button_style=''
)

def set_direction(b):
    clear_output()
    random_dir = latent_dirs[named_directions[select_variable.value][0]]
    start_layer = named_directions[select_variable.value][1]
    end_layer = named_directions[select_variable.value][2]
    print(start_layer, end_layer)
    out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})
    display(select_variable)
    display(ui, out)

random_dir = latent_dirs[named_directions[select_variable.value][0]]
start_layer = named_directions[select_variable.value][1]
end_layer = named_directions[select_variable.value][2]
seed = np.random.randint(0,100000)
style = {'description_width': 'initial'}

seed = widgets.IntSlider(min=0, max=100000, step=1, value=seed, description='Seed: ', continuous_update=False)
truncation = widgets.FloatSlider(min=0, max=2, step=0.1, value=0.7, description='Truncation: ', continuous_update=False)
distance = widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description='Distance: ', continuous_update=False, style=style)
scale = widgets.FloatSlider(min=0, max=10, step=0.05, value=1, description='Scale: ', continuous_update=False)

bot_box = widgets.HBox([seed, truncation, distance, scale])
ui = widgets.VBox([bot_box])
out = widgets.interactive_output(display_sample_pytorch, {'seed': seed, 'truncation': truncation, 'direction': fixed(random_dir), 'distance': distance, 'scale': scale, 'start': fixed(start_layer), 'end': fixed(end_layer)})

display(select_variable)
display(ui, out)

select_variable.observe(set_direction, names='value')



IndexError: ignored